In [13]:
import pandas as pd
import numpy as np
from datetime import datetime

In [14]:
data = pd.read_csv('tests.csv')
users = data.user_id.unique().astype(int)
users.sort()
users = users[1:len(users)]

In [16]:
total = []
psu = datetime(2016,11,28)
for user in users:
    user_tests = data[data['user_id']==user]
    total.append(len(user_tests))
    if len(data) > 1:
        filtered = user_tests[['subject_id','result','date']]
        filtered.columns = ['subject_id','score','date']
        subjects = filtered.subject_id.unique()
        for sub in subjects:
            filtered_sub = filtered[filtered['subject_id'] == sub]
            if len(filtered_sub) > 1:
                min_date = datetime.strptime(filtered.date.min().split(" ")[0],'%Y-%m-%d')
                max_date = datetime.strptime(filtered.date.max().split(" ")[0],'%Y-%m-%d')
                if min_date != max_date:
                    psu_day =(psu - min_date).days 
                    if psu_day < 0  or psu_day > 366:
                        continue
                    filtered_sub = filtered_sub[['score','date']]
                    filtered_sub.loc[:,'psu_day'] = pd.Series(np.array(list(map(lambda x: psu_day, range(filtered_sub.shape[0])))),index=filtered_sub.index)
                    filtered_sub['date'] = filtered_sub.apply((lambda x : (datetime.strptime(x['date'].split(" ")[0],'%Y-%m-%d')-min_date).days),axis=1)
                    filtered_sub.to_csv("data/{},{}.csv".format(user,sub),index=False)
                    
total = np.array(total)